# Particle Swarm Optimization

Leoni Mota Loris

## Swarm inteligence.

* Swarm intelligence is a type of artificial intelligence based on the collective behavior of self-organized systems.


* In general, such systems are made up of a population of simple agents interacting locally with one another and with the environment.


* Individual agents follow simple rules dictating how it should interact with the environment.


* Natural examples of swarm intelligence are *bacterial growth*, bird flocking, herds of animals, etc.


![assets/pso1.png](assets/pso1.png)

## Particle Swarm Optimization


* Models the problem to be optimized as an *n-dimensional* surface where one wants to find its global minimum.


* Each particle represents a possible solution (minimum) to the problem.


* The entire set of particles searches for an optimal solution by upgrading its generations. 


* But **it does not resample** its particles, unlike other population-based algorithms.

![](assets/pso2.png)

## Direct Applications

* Adaptive Antenna Arrays.


* Molecular Docking.

### Adaptive Antenna Arrays.
![](assets/array.png)

* Each antenna applies a gain and a phase-shift 


* Thus it is possible to place `Lobes` and `nulls` on the irradication pattern.


* In an adaptive array, the antennas need to be able to determine the optimal parameters in real time when a signal is detected.
